# **SpaceX  Falcon 9 First Stage Landing Prediction**


## Notebook: Data Analysis with SQL

## Objectives
#### - Create and connect to SQLite3 database
#### - Execute SQL queries
#### - Parse the table and convert it into a Pandas data frame


## Introduction

SpaceX has gained worldwide attention for a series of historic milestones. It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


In [27]:
# !pip install sqlalchemy==1.3.9

### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [28]:
# %load_ext sql

In [29]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cursor = con.cursor()

In [30]:
# !pip install -q pandas==1.1.5

In [31]:
# %sql sqlite:///my_data1.db

In [32]:
import pandas as pd
df = pd.read_csv("Spacex.csv")
df.to_sql("spacex", con, if_exists='replace', index=False,method="multi")

999

In [33]:
data = cursor.execute("SELECT * FROM spacex").fetchall()
column_names = [description[0] for description in cursor.description]  # Get the column names from the cursor

df = pd.DataFrame(data, columns=column_names)
df

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,06/04/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute)
1,12/08/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525.0,LEO (ISS),NASA (COTS),Success,No attempt
3,10/08/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt
4,03/01/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt
...,...,...,...,...,...,...,...,...,...,...
994,None,None,None,None,None,NaN,None,None,None,None
995,None,None,None,None,None,NaN,None,None,None,None
996,None,None,None,None,None,NaN,None,None,None,None
997,None,None,None,None,None,NaN,None,None,None,None


### Query 1

##### Names of the unique launch sites in the space mission


In [34]:
# unique_names

data = cursor.execute("select distinct(Launch_Site) from spacex where Launch_Site!='None'").fetchall()
column_names = ['Launch_Site']
unique_names = pd.DataFrame(data, columns= column_names)
unique_names

,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40



### Query 2


#####  Some records where launch sites begin with the string 'CCA' 


In [35]:
# unique_names
data = cursor.execute("select * from spacex where Launch_Site like 'CCA%' limit 5").fetchall()
column_names = [i[0] for i in cursor.description]
unique_names = pd.DataFrame(data, columns= column_names)
unique_names

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,06/04/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute)
1,12/08/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525.0,LEO (ISS),NASA (COTS),Success,No attempt
3,10/08/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt
4,03/01/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt


### Query 3




##### Total payload mass carried by boosters launched by NASA (CRS)


In [36]:
data = cursor.execute("select sum(PAYLOAD_MASS__KG_) from spacex where Payload like '%CRS%'").fetchall()
column_names = ['Total payload']
total_payload = pd.DataFrame(data,columns=column_names)
total_payload

,Total payload
0,111268.0


### Query 4




##### Average payload mass carried by booster version F9 v1.1


In [37]:
data = cursor.execute("select avg(PAYLOAD_MASS__KG_) from spacex where Booster_Version	= 'F9 v1.1'").fetchall()
column_names = ['Total payload']
avg_payload = pd.DataFrame(data,columns=column_names)
avg_payload

,Total payload
0,2928.4


### Query 5

##### Date when the first succesful landing outcome in ground pad was acheived.

In [38]:
data = cursor.execute("SELECT MIN(DATE) as 'Date' FROM spacex WHERE Landing_Outcome = 'Success (ground pad)'").fetchall()
column_names = [i[0] for i in cursor.description]
first_landing = pd.DataFrame(data,columns=column_names)
first_landing

,Date
0,01/08/2018


### Query 6

##### Names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [39]:
data = cursor.execute("SELECT Booster_Version FROM spacex WHERE Landing_Outcome = 'Success (drone ship)' and PAYLOAD_MASS__KG_ between 4000 and 6000").fetchall()
column_names = ['Booster Version']
first_landing = pd.DataFrame(data,columns=column_names)
first_landing

,Booster Version
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


### Query 7




##### Total number of successful and failure mission outcomes


In [40]:
data = cursor.execute("SELECT Mission_Outcome, COUNT(*) AS QTY FROM spacex GROUP BY Mission_Outcome ORDER BY Mission_Outcome").fetchall()
column_names = ['Booster Version', 'Quantity']
first_landing = pd.DataFrame(data,columns=column_names)
first_landing

,Booster Version,Quantity
0,None,898
1,Failure (in flight),1
2,Success,98
3,Success,1
4,Success (payload status unclear),1


### Query 8



##### Names of the booster_versions which have carried the maximum payload mass


In [41]:
df

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,06/04/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute)
1,12/08/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525.0,LEO (ISS),NASA (COTS),Success,No attempt
3,10/08/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt
4,03/01/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt
...,...,...,...,...,...,...,...,...,...,...
994,None,None,None,None,None,NaN,None,None,None,None
995,None,None,None,None,None,NaN,None,None,None,None
996,None,None,None,None,None,NaN,None,None,None,None
997,None,None,None,None,None,NaN,None,None,None,None


In [42]:
data = cursor.execute("SELECT DISTINCT Booster_Version FROM spacex WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM spacex) ORDER BY Booster_Version").fetchall()
column_names = ['Booster Version']
first_landing = pd.DataFrame(data,columns=column_names)
first_landing

,Booster Version
0,F9 B5 B1048.4
1,F9 B5 B1048.5
2,F9 B5 B1049.4
3,F9 B5 B1049.5
4,F9 B5 B1049.7
5,F9 B5 B1051.3
6,F9 B5 B1051.4
7,F9 B5 B1051.6
8,F9 B5 B1056.4
9,F9 B5 B1058.3


### Query 9


##### Listing of records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [43]:
data = cursor.execute("SELECT DISTINCT Booster_Version FROM spacex WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM spacex) ORDER BY Booster_Version").fetchall()
column_names = ['Booster Version']
first_landing = pd.DataFrame(data,columns=column_names)
first_landing

,Booster Version
0,F9 B5 B1048.4
1,F9 B5 B1048.5
2,F9 B5 B1049.4
3,F9 B5 B1049.5
4,F9 B5 B1049.7
5,F9 B5 B1051.3
6,F9 B5 B1051.4
7,F9 B5 B1051.6
8,F9 B5 B1056.4
9,F9 B5 B1058.3


In [44]:
data = cursor.execute('''SELECT
    substr(Date, 4, 2),
    Landing_Outcome,
    Booster_Version,
    Launch_Site
FROM
    spacex
WHERE
    Landing_Outcome = 'Failure (drone ship)'
    AND substr(Date, 7, 4) = '2015';
''').fetchall()
column_names = ['Month','Landing outcome','Booster version','Launch Site']
first_landing = pd.DataFrame(data,columns=column_names)
first_landing

,Month,Landing outcome,Booster version,Launch Site
0,10,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
1,04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Query 10

##### Ranking the count of  successful landing outcomes between the date 04-06-2010 and 20-03-2017 in descending order.


In [45]:
df

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,06/04/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute)
1,12/08/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525.0,LEO (ISS),NASA (COTS),Success,No attempt
3,10/08/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt
4,03/01/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt
...,...,...,...,...,...,...,...,...,...,...
994,None,None,None,None,None,NaN,None,None,None,None
995,None,None,None,None,None,NaN,None,None,None,None
996,None,None,None,None,None,NaN,None,None,None,None
997,None,None,None,None,None,NaN,None,None,None,None


In [46]:
df.Landing_Outcome.value_counts()

Landing_Outcome
Success                   38
No attempt                21
Success (drone ship)      14
Success (ground pad)       9
Controlled (ocean)         5
Failure (drone ship)       5
Failure                    3
Failure (parachute)        2
Uncontrolled (ocean)       2
Precluded (drone ship)     1
No attempt                 1
Name: count, dtype: int64

In [47]:
data = cursor.execute('''
SELECT 
    Landing_Outcome,
    COUNT(*) AS Success_Count
FROM
    spacex
WHERE
    DATE(SUBSTR(Date, 7, 4) || '-' || SUBSTR(Date, 4, 2) || '-' || SUBSTR(Date, 1, 2)) BETWEEN '2010-06-04' AND '2017-03-20'
    AND Landing_Outcome LIKE '%Success%'
GROUP BY
    Landing_Outcome
ORDER BY
    Success_Count DESC;

''').fetchall()
column_names = ['Landing Outcome','Quantity']
first_landing = pd.DataFrame(data,columns=column_names)
first_landing

,Landing Outcome,Quantity
0,Success (ground pad),5
1,Success (drone ship),5


In [48]:
df.head(10)

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,06/04/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute)
1,12/08/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525.0,LEO (ISS),NASA (COTS),Success,No attempt
3,10/08/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt
4,03/01/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt
5,29/09/2013,16:00:00,F9 v1.1 B1003,VAFB SLC-4E,CASSIOPE,500.0,Polar LEO,MDA,Success,Uncontrolled (ocean)
6,12/03/2013,22:41:00,F9 v1.1,CCAFS LC-40,SES-8,3170.0,GTO,SES,Success,No attempt
7,01/06/2014,22:06:00,F9 v1.1,CCAFS LC-40,Thaicom 6,3325.0,GTO,Thaicom,Success,No attempt
8,18/04/2014,19:25:00,F9 v1.1,CCAFS LC-40,SpaceX CRS-3,2296.0,LEO (ISS),NASA (CRS),Success,Controlled (ocean)
9,14/07/2014,15:15:00,F9 v1.1,CCAFS LC-40,OG2 Mission 1 6 Orbcomm-OG2 satellites,1316.0,LEO,Orbcomm,Success,Controlled (ocean)


## Created by Hrishikesh Reddy Papasani 
### Github: https://github.com/Hrishikesh-Papasani
### Contact: hrpapasani@gmail.com